# Lab 2 - FTP Over TCP
In this lab you will take the knowledge you learned during the previous lab about setting up and using TCP sockets in order to build a simple file transfer system that uses the Request-Reply protocol. You have been provided with a program structure for implementing your distributed system.

## 2.1 - Primer
In labs 2, 4, and 5, you will create different communication interfaces inheriting from the CommunicationInterface class. In this lab you will also need to implement an FTClient and FTServer. The intent is for a future-compatible design in which your FTClient and FTServer can be used interchangably with different communication interfaces using the same command and file transfer logic.

In this lab the FTClient and FTServer import the TCPFileTransfer class as the communication interface:
```
from EEE466Baseline.TCPFileTransfer import TCPFileTransfer as CommunicationInterface
```
In lab 4, this line looks like:
```
from EEE466Baseline.TCPFileTransfer import RUDPFileTransfer as CommunicationInterface
```

### CommunicationInterface
The client and server will communicate using a class which overrides the methods defined in the CommunicationInterface class (this is the TCPFileTransfer class for lab 2). This class provides a set of methods the client and server can call to perform a set of tasks (an API!). How these tasks are completed are abstracted away from the client and server through this *interface*.

The CommunicationInterface class contains stub functions to be overridden in the TCPFileTransfer class. As an example to illustrate this point, consider the terminalInterface class used for printing to the console shown below. The terminalInterface class has a function called printWithTimestamp. The commandLineWriter class inherits from the terminalInterface class in the mainProgram function. Here are good, better and incorrect examples:

In [1]:
#Good example - it maintains the interface for mainProgram to access commandLineWriter

import time

class terminalInterface(object):

    def printWithTimestamp(self, valueToPrint):
        print("TODO implement this function")

# commandLineWriter inherits terminalInterface and overrides printWithTimestamp
class commandLineWriter(terminalInterface):
    def printWithTimestamp(self, valueToPrint):
        print(time.ctime()+" "+str(valueToPrint))

class mainProgram(object):
    _printer = commandLineWriter()
    def run(self):
        self._printer.printWithTimestamp("program started")

In [2]:
mainProgram().run()
commandLineWriter().printWithTimestamp("program started")

Fri Sep 22 13:59:30 2023 program started
Fri Sep 22 13:59:30 2023 program started


In [8]:
#Better example - it maintains the interface for mainProgram to access commandLineWriter
#but it also uses terminalInterface to make it easier to make new alternatives to
#commandLineWriter by moving the shared requirement to get the time into the terminalInterface class.

import time

class terminalInterface(object):
    def printWithTimestamp(self, valueToPrint):
        self.printBoth(time.ctime(), valueToPrint)
    def printBoth(self, firstValToPrint, secondValToPrint):
        print ('TODO implement this function')

class commandLineWriter(terminalInterface):
    def printBoth(self, firstValToPrint, secondValToPrint):
        print(str(firstValToPrint)+" "+str(secondValToPrint))

# Velasco added this one
class commandLineWriterV2(terminalInterface):

    def printBoth(self, firstValToPrint, secondValToPrint):

        print("In 2.0, reverse values: " + str(secondValToPrint) + " " + str(firstValToPrint));

class mainProgram(object):
    _printer = commandLineWriter()
    def run(self):
        self._printer.printWithTimestamp("program started") # <-- allows for polymorphism when you change the object in _printer

        self._printer = commandLineWriterV2()
        self._printer.printWithTimestamp("Program started again.")

In [9]:
mainProgram().run()
# commandLineWriter().printWithTimestamp("program started")

Fri Sep 22 14:06:16 2023 program started
In 2.0, reverse values: Program started again. Fri Sep 22 14:06:16 2023


In [10]:
#Bad example because it breaks the interface between mainProgram and commandLineWriter
#this example will run in this particular case, but if someone else writes code to use a terminalInterface
#and they write commandLineWriter().printWithTimestamp("program started"), the console will print "#TODO implement this".

import time

class terminalInterface(object):
    def printWithTimestamp(self, valueToPrint):
        print("TODO implement this")

class commandLineWriter(terminalInterface):
    def printBoth(self, firstValToPrint, secondValToPrint):
        print(str(firstValToPrint)+" "+str(secondValToPrint))

    def getTime(self):
        return time.ctime()

class mainProgram(object):
    _printer = commandLineWriter()
    def run(self):
        self._printer.printBoth(self._printer.getTime(),"program started")

In [11]:
mainProgram().run()
commandLineWriter().printWithTimestamp("program started")

Fri Sep 22 14:06:45 2023 program started
TODO implement this


Note: see [link](http://blog.thedigitalcatonline.com/blog/2014/05/19/method-overriding-in-python/) for a discussion of method overriding in python. Done correctly you can write classes to work with CommunicationInterface and then override specific methods only to alter the manner in which those classes communicate.

### Implementation

The design of this lab is critical to success in later labs. If your design is well conceived, you may reuse a significant amount of code for lab 4 and lab 5. The crux of this problem is preparation. There are two primary challenges to consider:
1. What data will be sent. Format and size commands and files will look like to the server and the client. Both entities should share a data representations so that they can correctly parse received messages.
2. What logic should be in the client/server endpoints and what should be contained within the TCP interface.

It is *strongly* encouraged that you begin this lab by writing or sketching out what actions each class will be responsible for before beginning your implementation. You may also want to ask your instructor for feedback on your design. As long as the socket logic is limited to the interface you will meet the design requirements. However, the intent is to have a generic client and server that allow for transmission of files without getting specific on how messages are passed. This will mean easy integration with the interfaces introduced in future labs.

Note: You may consider altering the code in the CommunicationInterface/TCPFileTransfer classes (e.g., adding helper methods) as long as the predefined methods are used for interaction with the FTServer and FTClient. This is not required to successfully complete the labs.

## 2.2 - Requirements
Here are the design requirements for implementing your client, server, and, TCP interface. The classes are provided for you along with example files to test file transfer. 

### FTClient
- The FTClient will start up and present an input prompt to the user through which the user will be able to type commands and submit them by pressing enter;
- The FTClient module will not import from the socket package, all communications code will be encapsulated in the TCPFileTransfer module in the EEE466Baseline package;
- The FTClient will return to an input prompt after processing a command, unless it shuts down as a result of an error or a QUIT command;
- The FTClient must be capable of processing the following commands:
    1. `get,filename` - Example: `get,test.txt` - When this command is received, the FTClient will request the file specified from the server. If it is available in the server's Send folder, the server will send it to the client which will store it in the Client's Receive folder;
       - If the client requests a file that doesn't exist, the server will return an error indicating the file doesn't exist and the client will notify the user;
       - If the client has an existing file in its Receive folder with the same name as the requested file, the existing file will be overwritten by the newly downloaded version;
    2. `put,filename` - Examples: `put,test.txt`` - When this command is received, the FTClient will check its Send folder for the filename. If it exists, it will send an appropriate put command to the server followed by the file itself;
       - If the user specifies a file that does not exist, the client will notify the user;
    3. `quit` - When this command is received the client will send a message to the server to instruct it to shut down, wait for the server to acknowledge the message, and then exit;
- The client and server should be able to transfer multiple files in a single session (you should be able to put or get multiple files before closing the connection);

### FTServer
- The FTServer will start up, bind to a port, and listen for incoming connections;
- The FTServer module will not import from the socket package, all communications code will be encapsulated in the TCPFileTransfer module in the EEE466Baseline package;
- The FTServer will only accept one client connection at a time;
- When the FTServer accepts a connection, it will respond to the following commands from the FTClient:
    - `get,filename` - Example: `get,test.txt` - When the FTServer receives a get command it will look for the specified file in its Send folder, if the file does not exist it will return an error indicating the requested file doesn't exist, if the file exists it will be transmitted to the client;
    - `put,filename` - Example: `put,test.txt` - When the server receives a put command it will prepare to receive the specified file and then save the received file in its Receive folder, if there is already a file there with the same name it will be overwritten;
- When the FTServer receives a `quit` message, it will properly shut down;

### TCPFileTransfer
The TCPFileTransfer class inherits from the communication CommunicationInterface class and must be used to implement interface objects for this lab.
- The TCPFIleTransfer class inherits from the CommunicationInterface class;
- The TCPFileTransfer class may import from the socket package;
- The TCPFileTransfer class will never use a hard coded buffer size of 1. This is for two reasons:
    1. When a buffer size of 1 is used, it negates having to learn how to deal with streams of data which is an important part of using TCP;
    2. Every call to send or recv requires a context switch to the Operating System, which in computer terms is very expensive. High performance system programmers go to great lengths to make as few of these calls as possible (without making other inappropriate engineering decisions like using way too much RAM). A buffer size of 1 is inappropriate for a system of any size even if a system uses fixed length 1 byte messages.
- Due to significant (imaginary) hardware constraints, the client and server can only buffer 1024 bytes. Therefore, sends and receives performed by the interface must not exceed this size. Files can be much larger than this, and must be split into multiple sends/receives from the communication channel.

Note: The client and server contain example functions that may guide you in your design. These functions and their parameters are completely optional.

# 2.3 - Testing
An important aspect of distributed system development is to properly test your implementation. Testing and debugging distributed applications can be quite a bit more difficult than testing a single application. This is partly due to the fact that you are now debugging two programs simultaneously, but also because the only means of communicating between these programs is to send messages. If your program doesn't work as expected, it may be worthwhile to use Wireshark or tcpdump to view the messages that are (or aren't) being sent between your client and server. 

You will be marked on the ability of your system to handle erroneous inputs, such as:
- Invalid command, extra blank spaces or commas
- File requested doesn't exist (put or get)

It is recommended that you create a test set for proper functionality and edge cases and include them in your code. By default, your code should run as a command line prompt so any test cases should be commented or provided as a prompt options.

The text files you transfer should be of different sizes, and be able to transmit in either direction. Explain your testing in your submitted code, and include any test files as needed. Reminder: you don't *need* to include testing information in your submission but if your code fails, clearly documented testing can provide partial marks.

# 2.5 - Submission
Your objective for this lab is to implement the following files using the requirements listed in part 2.2:
1. FTClient.py
2. FTServer.py
3. TCPFileTransfer.py

All files included in this lab (i.e., your project) are to be submitted in a zip file (see lab_report_template.ipynb for instructions). Use the lab report template to reference the files that you modified and provide instructions/clarification as required.